In [1]:
#pip install deepface

In [1]:
import cv2

In [12]:
import cv2
import numpy as np
from sklearn.metrics import pairwise
bck=None
acc_weight=0.5
roi_top=20
roi_bot=200
roi_rig=400
roi_left=700

In [13]:
def calc(f,acc):
    global bck
    if bck is None:
        bck=f.copy().astype('float')
        return None
    cv2.accumulateWeighted(f,bck,acc)
def seg(f,threshold=25):
    dif=cv2.absdiff(bck.astype('uint8'),f)
    r,thresh=cv2.threshold(dif,threshold,255,cv2.THRESH_BINARY)
    contours,hier=cv2.findContours(thresh.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    if len(contours)==0:
        return None
    else:
        hand_seg=max(contours,key=cv2.contourArea)
        return (thresh,hand_seg)

In [14]:
def fingers(thresh,hand_seg):
    conv_hull=cv2.convexHull(hand_seg)
    top=tuple(conv_hull[conv_hull[:,:,1].argmin()][0])
    bot=tuple(conv_hull[conv_hull[:,:,1].argmax()][0])
    left=tuple(conv_hull[conv_hull[:,:,0].argmin()][0])
    right=tuple(conv_hull[conv_hull[:,:,0].argmax()][0])
    cX=(left[0]+right[0])//2
    cY=(top[1]+bot[1])//2
    #point1 = np.array([cX, cY]).reshape(1, -1)
    #point2 = np.array([left, right, top, bot]).reshape(1, -1)
    #dist= pairwise.euclidean_distances(point1, point2)[0, 0]
    dist=pairwise.euclidean_distances(X=[(cX,cY)],Y=[left,right,top,bot])[0]
    max_dist=dist.max()
    r=int(0.7*max_dist)
    c=(2*np.pi*r)
    croi=np.zeros(thresh.shape[:2],dtype='uint8')
    cv2.circle(croi,(cX,cY),r,255,10)
    croi=cv2.bitwise_and(thresh,thresh,mask=croi)
    contours,hier=cv2.findContours(croi.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    count=0
    for i in contours:
        (x,y,w,h)=cv2.boundingRect(i)
        out=(cY+(cY*0.15))>(y+h)
        limit=((c*0.15)>i.shape[0])
        if out and limit:
            count+=1
    return count


In [15]:
cam=cv2.VideoCapture(0)
numf=0
while True:
    r,f=cam.read()
    f_copy=f.copy()
    roi=f[roi_top:roi_bot,roi_rig:roi_left]
    gray=cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    gray=cv2.GaussianBlur(gray,(7,7),0)
    if numf<60:
        calc(gray,acc_weight)
        if numf<=59:
            cv2.putText(f_copy,'WAIT... LOADING',(200,200),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            cv2.imshow('Finger Count',f_copy)
    else:
        hand=seg(gray)
        if hand is not None:
            thresh,hand_seg=hand
            cv2.drawContours(f_copy,[hand_seg+(roi_rig,roi_top)],-1,(255,0,0),5)
            count=fingers(thresh,hand_seg)
            cv2.putText(f_copy,str(count),(70,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
            cv2.imshow('Threshold',thresh)
    cv2.rectangle(f_copy,(roi_left,roi_top),(roi_rig,roi_bot),(0,0,255),5)
    numf+=1
    cv2.imshow('Finger Count',f_copy)
    k=cv2.waitKey(1) & 0xFF
    if k==27:
        break
cam.release()
cv2.destroyAllWindows()
        